In [4]:
!pip install accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 72.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 126.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
#modified from crowsonkb's code 
import argparse
import json
from pathlib import Path
import sys

import accelerate
import peft
import torch
from torch import optim
from torch.utils import data
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import trange, tqdm

In [6]:
print = tqdm.external_write_mode()(print)


In [7]:
def batch_to_tensors(batch, device="cpu"):
    seq_len = max(len(x) for x in batch)
    input_ids = torch.zeros(len(batch), seq_len, dtype=torch.long, device=device)
    attention_mask = torch.zeros(len(batch), seq_len, dtype=torch.long, device=device)
    for i, x in enumerate(batch):
        input_ids[i, : len(x)] = torch.tensor(x, dtype=torch.long, device=device)
        attention_mask[i, : len(x)] = 1
    return input_ids, attention_mask

In [15]:
batch_size = 4
bits = 16 #quantization bits
dataset = "test-dataset" #path
dropout = 0.2 #dropout rate
epoches = 1
lr = 1e-4 #learning rate
model_name = "meta-llama/Llama-2-7b-chat-hf" #model name
output = "model"
rank = 8 #lora rank

In [9]:
accelerator = accelerate.Accelerator(
    mixed_precision="bf16", gradient_accumulation_steps=1
)

In [11]:
device = accelerator.device
is_main = accelerator.is_main_process
print0 = accelerator.on_main_process(print)

In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=args.bits == 4,
    load_in_8bit=args.bits == 8,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

NameError: name 'args' is not defined